<a href="https://colab.research.google.com/github/rajivsam/window_size_experiments/blob/main/notebooks/window_size_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
This notebook contains an implementation of the algorithm described in [this paper from Eamon Keogh's lab]( https://kdd-milets.github.io/milets2021/papers/MiLeTS2021_paper_9.pdf ). The implementation is illustrated for two (randomly picked) datasets. This is based on the template provided in this [issue](https://github.com/TDAmeritrade/stumpy/issues/1062).

### Get the data from the git repo

In [1]:
!git clone https://github.com/rajivsam/window_size_experiments/

fatal: destination path 'window_size_experiments' already exists and is not an empty directory.


### Install the required libraries

In [2]:
fp = "/content/window_size_experiments/data/data_phase2/001_UCR_Anomaly_35000.txt"

In [3]:
!pip install -r /content/window_size_experiments/requirements.txt

### Import the required libraries


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import os

import plotly.graph_objects as go

### Provision the data structures needed for the experiment
The crux of the method is capture the behavior of the residuals and the window size as described in section 4.1 of the [paper]( https://kdd-milets.github.io/milets2021/papers/MiLeTS2021_paper_9.pdf). Understanding this sketch requires a review and understanding of the paper and this section in particular.

In [5]:
movingAvgResiduals = []
window_sizes = []

### Code for the window size experiment

In [6]:
def zscore(ts):
    ts = (ts - np.nanmean(ts))/(np.nanstd(ts))
    return ts

def movmean(ts, w):
    """
    # faster solution of moving ave
    moving_avg = np.cumsum(ts, dtype=float)
    moving_avg[w:] = moving_avg[w:] - moving_avg[:-w]
    return moving_avg[w-1:] / w
    """


    moving_avg = np.cumsum(ts, dtype=float)
    moving_avg[w:] = moving_avg[w:] - moving_avg[:-w]
    return moving_avg[w-1:] / w



def original_finding_window_size(ts, start, end, step):
    """
    finidng appropriate window size using movemean
    """
    threshold = float('inf')
    all_averages = []

    #step size is now parametrized
    for w in range(start, end, step):

        movingAvg = np.array(movmean(ts, w))

        all_averages.append(movingAvg)
        window_sizes.append(w)



    for i, w in enumerate(window_sizes):
        moving_avg = all_averages[i][:len(all_averages[-1])]

        movingAvgResidual = np.log(abs(moving_avg - (moving_avg).mean()).sum())

        movingAvgResiduals.append(movingAvgResidual)

    b = (np.diff(np.sign(np.diff(movingAvgResiduals))) > 0).nonzero()[0] + 1  #local min , RS: This identifies the local minima
    reswin = []
    try:
        for i in range(3):
            reswin.append(window_sizes[b[i]]/ (i+1))
        reswin = np.array(reswin)
        winTime = 0.8 * reswin[0] + 0.15 * reswin[1] + 0.05 * reswin[2] # RS: This seems to be some weighting of the first 3 local minima
#         winTime = np.mean(reswin)
        conf = np.std(reswin)/  np.sqrt(3)
    except:
        winTime = window_sizes[b[0]] # RS: If you don't have 3 minima, default to the first, the first is a good estimate anyway.
        conf = np.nan
#     kn =  windowLocator(window_sizes, movingAvgResiduals) # window size (the reason I choose kn is because when I wanted to be the same name as kneelocator function)
#     movingAvgResiduals = list(np.array(movingAvgResiduals[:-1]) - np.array(movingAvgResiduals[1:]))

    return winTime, conf, reswin

### Evaluate method on dataset #1

In [7]:
fp = "/content/window_size_experiments/data/data_phase2/001_UCR_Anomaly_35000.txt"
df = pd.read_csv(fp)
ts = zscore(df.values)

In [8]:
# using s and e values in the listing for ALgorithm 1 in the paper.
start = 10
end = 1000
step = 10
winTime, conf, reswin= original_finding_window_size(ts, start, end, step)

## What is the window size returned?
1. The first local minima, the first element of the array ```reswin``` is a good estimate.
2. If you have multiple local minima, the weighted average in ```winTime``` is a good estimate

In [9]:
reswin[0]

210.0

In [10]:
dfp = pd.DataFrame.from_records({"window_size": window_sizes, "moving_dist": movingAvgResiduals})

### Create the plot from the experiment to determine the window size

In [11]:
fig = px.line(dfp, x="window_size", y="moving_dist", title='Moving Dist vs Window Size',  symbol='moving_dist')
fig.show()

### Repeat the process for the second time series

In [12]:
fp = "/content/window_size_experiments/data/data_phase2/007_UCR_Anomaly_4000.txt"
df = pd.read_csv(fp)
ts = zscore(df.values)

In [13]:
movingAvgResiduals = []
window_sizes = []
# using s and e values in the listing for ALgorithm 1 in the paper.
start = 10
end = 100
step = 1
winTime, conf, reswin= original_finding_window_size(ts, start, end, step)

In [14]:
reswin[0]

25.0

In [15]:
dfp = pd.DataFrame.from_records({"window_size": window_sizes, "moving_dist": movingAvgResiduals})
fig = px.line(dfp, x="window_size", y="moving_dist", title='Moving Dist vs Window Size',  symbol='moving_dist')
fig.show()